In [ ]:
import numpy as np
import pandas as pd

# Data Load

In [ ]:
data = pd.read_csv('newdata.csv', encoding='cp949')
data

# Data Preprocessing

In [ ]:
# Scaling

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
train_set_scaled = scaler.fit_transform(train_set.iloc[:, [0, 1, 2, 3, 4, 9, 10, 11]])
test_set_scaled = scaler.transform(test_set.iloc[:, [0, 1, 2, 3, 4, 9, 10, 11]])

train_set.iloc[:, [0, 1, 2, 3, 4, 9, 10, 11]] = train_set_scaled
test_set.iloc[:, [0, 1, 2, 3, 4, 9, 10, 11]] = test_set_scaled

train_set = train_set.values
test_set = test_set.values

In [ ]:
# Creating a data structure with 432 timesteps and 4 outputs

X_train = []
y_train = []
step = 432   # 432개의 input으로 특정 시점의 수위 예측
features = 8   # 독립변수 개수

for i in range(step, len(train_set)):
    X_train.append(train_set[i-step:i, [0, 1, 2, 3, 4, 9, 10, 11]])
    y_train.append(train_set[i, 5:9])

X_train, y_train = np.array(X_train), np.array(y_train)
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Reshaping

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], features))
print(X_train.shape)
print(y_train.shape)

# Model

## Model 1 : Baseline

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, TimeDistributed
from keras.layers import Conv1D, AveragePooling1D

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2]))))
model.add(Dropout(0.2))

model.add(Dense(units=4, activation='relu'))

model.compile(optimizer='adam', loss=tf.keras.metrics.mean_squared_error, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=256)

## Model 2 : Layer 추가

In [ ]:
# Batch Normalization -> Dropout -> Activation -> Pooling

model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=4))

model.compile(optimizer='adam', loss=tf.keras.metrics.mean_squared_error, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=256)